In [2]:
import os
os.chdir('C:/myroom/')
print(os.getcwd())

C:\myroom


In [3]:
#데이터 핸들링
import pandas as pd
import numpy as np

#전처리
from sklearn.preprocessing import StandardScaler, MinMaxScaler , RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from statsmodels.stats.outliers_influence import variance_inflation_factor

#모델
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

#하이퍼 파라미터 튜닝: 보통 랜덤서치로 대략 파악한 다음 그리드 서치로 미세조정
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV


#평가
from sklearn.metrics import f1_score

In [4]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
#pd.set_option('display.max_rows',None)
#pd.set_option('display.max_colwidth', None)

train= pd.read_csv("./nlp-getting-started/train.csv")
test= pd.read_csv("./nlp-getting-started/test.csv")
train.shape, test.shape
# ((7613, 5), (3263, 4))

train['target'].value_counts()
#1:실제 발생, 0: 발생하지 않음

target
0    4342
1    3271
Name: count, dtype: int64

In [5]:
X=train.iloc[:,1:4]
y=train['target']
X.head()

,keyword,location,text
0,NaN,NaN,Our Deeds are the Reason of this #earthquake M...
1,NaN,NaN,Forest fire near La Ronge Sask. Canada
2,NaN,NaN,All residents asked to 'shelter in place' are ...
3,NaN,NaN,"13,000 people receive #wildfires evacuation or..."
4,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...


In [5]:
!pip install transformers

In [6]:
!pip install ipywidgets

In [34]:
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import DataLoader
import torch

# 1. 트위터 특화 BERT 로드
model_name = "cardiffnlp/twitter-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 2. 문장 리스트를 CLS 임베딩으로 벡터화하는 함수
def get_cls_batch(texts, batch_size=16, max_len=128):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # 1. 텍스트 전처리: HTTP URL 제거 및 공백 제거
    # 텍스트가 아닌 숫자나 다른 타입도 고려하여 처리
    texts = [str(text).replace("http://", "").replace("https://", "").strip() for text in texts]
    
    # 2. DataLoader는 텍스트 리스트로만 동작
    all_embeddings = []
    
    # 텍스트 리스트를 DataLoader로 변환
    dataloader = DataLoader(texts, batch_size=batch_size, shuffle=False)

    for batch in dataloader:
        # 3. 배치가 여전히 텍스트 리스트이므로, 토크나이저로 텍스트를 토크나이징
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=max_len).to(device)

        # 4. 모델에 입력하고 CLS 임베딩 추출
        with torch.no_grad():
            outputs = model(**inputs)
            cls_embeddings = outputs.last_hidden_state.mean(dim=1)  # CLS 토큰 임베딩

        all_embeddings.append(cls_embeddings.cpu())

    return torch.cat(all_embeddings,dim=0)


In [35]:
vector3 = get_cls_batch(X['text'])

In [36]:
type(vector3)

torch.Tensor

In [37]:
vector3.shape

torch.Size([7613, 768])

In [38]:
import torch.nn as nn

mlp = nn.Sequential(
    nn.Linear(768,256),
    nn.ReLU(),
    nn.Dropout(0.1),
    nn.Linear(256,1),
)




In [39]:
output= mlp(vector3)
output.shape #torch.Size([7613, 1])

torch.Size([7613, 1])

In [40]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=1e-4)

In [53]:
X_train, X_val, y_train, y_val = train_test_split(vector3, y.values, test_size=0.2, random_state=42)

In [54]:
y_train = torch.tensor(y_train).float().unsqueeze(1)
y_val = torch.tensor(y_val).float().unsqueeze(1)
y_train.shape, y_val.shape

(torch.Size([6090, 1]), torch.Size([1523, 1]))

In [60]:
mlp.train()
for epoch in range(30):
    optimizer.zero_grad()
    outputs = mlp(vector3)
    loss = loss_fn(outputs,targets)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}: loss={loss.item():.4f}")

Epoch 1: loss=0.3985
Epoch 2: loss=0.3993
Epoch 3: loss=0.3986
Epoch 4: loss=0.3977
Epoch 5: loss=0.3980
Epoch 6: loss=0.3986
Epoch 7: loss=0.3983
Epoch 8: loss=0.3984
Epoch 9: loss=0.3982
Epoch 10: loss=0.3980
Epoch 11: loss=0.3982
Epoch 12: loss=0.3985
Epoch 13: loss=0.3981
Epoch 14: loss=0.3973
Epoch 15: loss=0.3981
Epoch 16: loss=0.3975
Epoch 17: loss=0.3971
Epoch 18: loss=0.3971
Epoch 19: loss=0.3974
Epoch 20: loss=0.3971
Epoch 21: loss=0.3968
Epoch 22: loss=0.3972
Epoch 23: loss=0.3973
Epoch 24: loss=0.3970
Epoch 25: loss=0.3971
Epoch 26: loss=0.3975
Epoch 27: loss=0.3968
Epoch 28: loss=0.3961
Epoch 29: loss=0.3962
Epoch 30: loss=0.3965


In [57]:
probs = torch.sigmoid(outputs).squeeze() 
probs.shape

torch.Size([6090])

In [58]:
preds = (probs >= 0.455).long().cpu().numpy()
true = targets.squeeze().long().cpu().numpy()

In [59]:
f1 = f1_score(true,preds)
f1

ValueError: Found input variables with inconsistent numbers of samples: [7613, 6090]